In [1]:
import pandas as pd
import py4cytoscape as p4c
import time
import requests

In [2]:
p4c.__version__

'1.6.0'

In [3]:
p4c.cytoscape_ping()

You are connected to Cytoscape!


'You are connected to Cytoscape!'

In [52]:
def create3Dnetwork(networkLayers, stylexml):
    layerTable = pd.read_table(networkLayers, header=None, sep="\t")
    networkSUID = layerTable.apply(importLayer, axis=1)
    layerTable = pd.concat([layerTable, networkSUID], axis=1)
    nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)
    layeredNodes = getLayeredNodes(nodetables)
    edgetables = layerTable.apply(getEdgeTableWithLayerinfo, axis=1)
    layeredEdges = pd.concat(edgetables.tolist())
    # layeredNodes.to_csv("layeredNodes.csv")
    # layeredEdges.to_csv("layeredEdges.csv")
    p4c.commands_post('cy3d set renderer')
    suID = p4c.create_network_from_data_frames(layeredNodes.astype(str), layeredEdges.astype(str))
    setTransomicStyle(stylexml, suID)
    return suID

In [47]:
def importLayer(row: pd.Series) -> int:
    #print(row.array)
    #getKgml(row.array[1])
    r = requests.get("https://rest.kegg.jp/get/" + row.array[1] + "/kgml")
    kgml = row.array[1] + ".xml"
    f = open(kgml, "w")
    f.write(r.text)
    f.close()    
    print("Importing " + kgml)
    res = p4c.import_network_from_file(kgml)
    time.sleep(3)
    xy = p4c.get_table_columns(table='node', columns=['KEGG_NODE_X', 'KEGG_NODE_Y'])
    xy = xy.rename(columns={"KEGG_NODE_X": "x_location", "KEGG_NODE_Y": "y_location"})
    p4c.load_table_data(xy, table_key_column="SUID")
    if row.array[3] == "true":
        print("Creating a node at each midpoint of the edge...")
        createMidnodes(res['networks'][0])
    return res['networks'][0]

In [ ]:
def createMidnodes(networkSUID):
    et = p4c.get_table_columns(table='edge', network=networkSUID)
    message("getting node positions...")
    xyloc = p4c.get_table_columns(table='node', network=networkSUID, columns=['SUID', 'x_location', 'y_location'])
    message("getting edge information...")
    einfo = p4c.get_edge_info(et['SUID'].tolist(), network=networkSUID)
    midxydf = [getMidLoc(x, xyloc)] for x in einfo]
    midxydf = pd.concat(midxydf)
    midxydf = midxydf.rename(columns={"x_location": "x_location", "y_location": "y_location"})
    midxydf = midxydf.drop_duplicates()
    message("creating midnodes...")
    p4c.create_nodes_from_data_frame(midxydf, network=networkSUID)
    message("setting node positions...")
    p4c.load_table_data(midxydf, table_key_column="SUID")
    message("done")
    


In [ ]:
def getMidLoc(elem, xyloc):
    sourcexy = xyloc[xyloc['SUID'] == elem['source'], ]
    targetxy = xyloc[xyloc['SUID'] == elem['target'], ]
    midx_location = (int(sourcexy['x_location'].values[0]) + int(targetxy['x_location'].values[0])) / 2
    midy_location = (int(sourcexy['y_location'].values[0]) + int(targetxy['y_location'].values[0])) / 2
    return pd.Series({'x_location': midx_location, 'y_location': midy_location})

In [54]:
et = p4c.get_table_columns(table='edge', network=213631)

Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312


In [60]:
xyloc = p4c.get_table_columns(table='node', network=213631, columns=['SUID', 'x_location', 'y_location'])

In [61]:
xyloc

,SUID,x_location,y_location
221187,221187,1746,1004
221184,221184,871,971
221190,221190,2140,1120
221193,221193,1892,748
221199,221199,3201,1027
...,...,...,...
221169,221169,1800,1004
221175,221175,1295,971
221172,221172,1344,971
221178,221178,1247,971


In [67]:
int(xyloc[xyloc['SUID'] == 221187]['x_location'].values[0])

1746

In [58]:
einfo = p4c.get_edge_info(et['SUID'].tolist(), network=213631)

In [59]:
einfo

[{'source': 223332,
  'target': 223335,
  'SUID': 227331,
  'shared name': 'rn:R04867',
  'shared interaction': 'reversible',
  'name': 'rn:R04867',
  'selected': False,
  'interaction': 'reversible',
  'KEGG_REACTION_TYPE': 'reversible',
  'KEGG_REACTION_GENE': 'rno:100359816 rno:100361710 rno:100362634 rno:102550344 rno:289372 rno:297333 rno:299521 rno:302553 rno:303547 rno:310638 rno:311968 rno:312299 rno:313678 rno:315606 rno:316013 rno:361798 rno:362078 rno:362831 rno:364785 rno:365155 rno:498295 rno:502710 rno:689883'},
 {'source': 220059,
  'target': 220062,
  'SUID': 225282,
  'shared name': 'rn:R05976',
  'shared interaction': 'reversible',
  'name': 'rn:R05976',
  'selected': False,
  'interaction': 'reversible',
  'KEGG_REACTION_TYPE': 'reversible',
  'KEGG_REACTION_GENE': 'rno:192275 rno:25596 rno:290783 rno:313648 rno:363160 rno:500972 rno:64701'},
 {'source': 223329,
  'target': 223332,
  'SUID': 227328,
  'shared name': 'rn:R04866',
  'shared interaction': 'reversible',


In [6]:
def getNodeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    nt = p4c.get_table_columns(table="node", network=row.array[4])
    nt['z_location'] = row.array[2]
    nt['LAYER_INDEX'] = row.array[0]
    return nt

In [7]:
def getLayeredNodes(nodetables: pd.DataFrame) -> pd.DataFrame:
    nodetable3d = pd.concat(nodetables.array)
    return nodetable3d.rename(columns={"SUID": "id"})

In [8]:
def getEdgeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    et = p4c.get_table_columns(table="edge", network=row.array[4])
    #return et
    print("Getting edge info. This function is kinda slow...")
    ei = p4c.get_edge_info(et["SUID"].tolist(), network=row.array[4])
    print("Finished getting edge info.")
    et['source'] = list(map(lambda x: x['source'], ei))
    et['target'] = list(map(lambda x: x['target'], ei))
    et['LAYER_INDEX'] = row.array[0]
    return et

In [39]:
def setTransomicStyle(xml, suid) -> None:
    #xml is must be a local file path
    stylename = p4c.import_visual_styles(xml)
    res = p4c.set_visual_style(stylename[0], network = suid)
    print(stylename[0] + " " + res['message'])

In [53]:
suid = create3Dnetwork("https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv", "yugi2014.xml")

Importing rno04910.xml
Importing rno01100.xml
Importing rno01100.xml
Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312
Getting edge info. This function is kinda slow...
Finished getting edge info.
Applying default style...
Applying preferred layout
yugi2014 Visual Style applied.
